In [29]:
from db import call_db_json
import pandas as pd
import numpy as np

In [44]:
current_HK_sql = f"""
    SELECT
        order_number, reference_number, item_description, product_family, product_model,
        product_variant, product_length, product_block_count, order_quantity, order_status,
        order_status, order_customer_name, order_scheduled_due, printed_due,
        (
            SELECT array_agg(facility_id ORDER BY mp.operation_sequence)
            FROM manufacturing_order_processes mp
            WHERE mp.order_number = mo.order_number
            AND mp.operation_status != '40'
        ) AS facilities
        from manufacturing_orders mo
        WHERE order_status IN (10, 40)
            AND printed_due >= CURRENT_DATE - 90
            AND reference_number LIKE 'HK%'
            AND order_class IN (11, 12, 13, 21)
            AND order_release_code = 5
            AND product_family NOT IN ('S-UNIT', 'GL', 'KR', 'TY')
        ORDER BY printed_due
    """
current_HK = pd.DataFrame(call_db_json(curren_HK_sql))

In [45]:
current_HK.columns


Index(['order_number', 'reference_number', 'item_description',
       'product_family', 'product_model', 'product_variant', 'product_length',
       'product_block_count', 'order_quantity', 'order_status',
       'order_customer_name', 'order_replied_due', 'order_scheduled_due',
       'order_original_due', 'order_scheduled_shipping_date', 'promise_date',
       'printed_due', 'facilities'],
      dtype='object')

In [46]:
current_HK

,order_number,reference_number,item_description,product_family,product_model,product_variant,product_length,product_block_count,order_quantity,order_status,order_customer_name,order_replied_due,order_scheduled_due,order_original_due,order_scheduled_shipping_date,promise_date,printed_due,facilities
0,M200556,HK310434,SHS15V2SSFME+528LF,SHS,15,V,528.0,2,10.0,40,B.E. AEROSPACE (UKNI),2023-11-30,2023-11-29,2023-11-30,2023-11-30,None,2023-09-28,"[J0045, K0010, K0020]"
1,M252803,HK342281,HSR15LR2C1FME+548LPF-II,HSR,15,LR,548.0,2,14.0,10,TECHNICO (COMP),2023-10-31,2023-10-31,2023-10-31,2023-10-31,None,2023-09-28,"[J0040, J0050, J0060, J0070, K0010, K0020]"
2,M260346,HK344862,HSR15LR2C1FME+548LPF-II,HSR,15,LR,548.0,2,14.0,40,TECHNICO (COMP),2023-11-22,2023-11-22,2023-10-31,2023-11-22,None,2023-09-28,"[J0040, J0070, K0010, K0020]"
3,M258125,HK344859,HSR25LR2C1FME+605LPF-II,HSR,25,LR,605.0,2,6.0,40,TECHNICO (COMP),2023-11-28,2023-11-28,2023-11-03,2023-11-28,None,2023-10-03,"[J0040, J0050, J0060, J0070, K0010, K0020]"
4,M252895,HK342107,HSR25LR2C1FME+605LPF-II,HSR,25,LR,605.0,2,8.0,10,TECHNICO (COMP),2023-11-03,2023-11-03,2023-10-31,2023-11-03,None,2023-10-03,"[J0040, J0050, J0060, J0070, K0010, K0020]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7221,M251807,HK341803,SR15W2SS+1600L,SR,15,W,1600.0,2,10.0,10,FLEXTRONICS INTL EUROPE B.V.,2024-10-15,2024-10-15,2024-10-15,2024-10-15,None,2024-10-15,"[J0045, K0010, K0020]"
7222,M251806,HK341803,SR15W2SS+1600L,SR,15,W,1600.0,2,10.0,10,FLEXTRONICS INTL EUROPE B.V.,2024-10-15,2024-10-15,2024-10-15,2024-10-15,None,2024-10-15,"[J0045, K0010, K0020]"
7223,M251805,HK341803,SR15W2SS+1600L,SR,15,W,1600.0,2,5.0,10,FLEXTRONICS INTL EUROPE B.V.,2024-10-15,2024-10-15,2024-10-15,2024-10-15,None,2024-10-15,"[J0045, K0010, K0020]"
7224,M251811,HK341803,SR15W2SS+1600L,SR,15,W,1600.0,2,10.0,10,FLEXTRONICS INTL EUROPE B.V.,2024-10-15,2024-10-15,2024-10-15,2024-10-15,None,2024-10-15,"[J0045, K0010, K0020]"


In [27]:
finished_operation_status = '40'
not_started_operation_status = '10'

def check_miss_scaned(test):    
    arr = np.array(test)
    finished_indices = np.where(arr == finished_operation_status)[0]
    not_started_indices = np.where(arr == not_started_operation_status)[0]
    try:
        last_finished_index = finished_indices[-1]
        not_started_first_index = not_started_indices[0]
        if not_started_first_index < last_finished_index:
            return True
        else:
            return False
    except:
        print(test)
        return False
processing['skipped_scan'] = processing['operation_statuses'].apply(check_miss_scaned)

['10', '10', '10']
['10', '10', '10', '10', '10', '10']
['10', '10', '10', '10', '10', '10']
['40', '40', '40', '40', '40', '30']
['10', '10']
['10', '10', '10']
['10', '10', '10', '10']
['10', '10', '10', '10']
['10', '10', '10', '10']
['10', '10', '10', '10']
['10', '10', '10', '10']
['10', '10', '10', '10']


In [28]:

processing[processing['skipped_scan']]

,order_number,product_family,product_model,item_description,facility_ids,order_sequences,operation_statuses,skipped_scan
31,M246784,SHS,30,SHS30R C1F BLOCK A0,"[E0010, E0020, F0020, H0010, J0050, J0025, J00...","[0010, 0020, 0030, 0040, 0042, 0043, 0045, 005...","[40, 40, 40, 40, 10, 10, 10, 40, 40, 30, 10]",True
